<a href="https://colab.research.google.com/github/DaylaHoang/TMDb_API_Call/blob/main/TMDb_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import time  # To handle API rate limits

In [ ]:
API_KEY = "728c7b4f5730549db84b7cafe2e0d30c"
BASE_URL = "https://api.themoviedb.org/3"

In [ ]:
def fetch_movies(max_pages=500):
    movies_list = []

    for page in range(1, max_pages + 1):  # Fetch up to max_pages (default 500)
        url = f"{BASE_URL}/movie/popular?api_key={API_KEY}&language=en-US&page={page}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            for movie in data['results']:
                movie_element = {
                    'movie_id': movie['id'],
                    'title': movie['title'],
                    'release_date': movie.get('release_date', None),
                    'vote_average': movie.get('vote_average', None),
                    'vote_count': movie.get('vote_count', None),
                    'popularity': movie.get('popularity', None),
                    'overview': movie.get('overview', None),
                    'poster_url': f"https://image.tmdb.org/t/p/w500{movie['poster_path']}" if movie.get('poster_path') else None
                }
                movies_list.append(movie_element)
        else:
            print(f"⚠️ Error {response.status_code} on page {page}: {response.text}")
            break  # Stop fetching if there's an API error

        time.sleep(0.5)  # Sleep to avoid hitting API limits

        if page % 50 == 0:  # Print progress every 50 pages
            print(f"Fetched {len(movies_list)} movies so far...")

    return movies_list

In [ ]:
def fetch_movie_details(movie_id):
    url = f"{BASE_URL}/movie/{movie_id}?api_key={API_KEY}&language=en-US"
    response = requests.get(url)

    if response.status_code == 200:
        movie = response.json()
        return {
            'budget': movie.get('budget', None),
            'revenue': movie.get('revenue', None),
            'runtime': movie.get('runtime', None),
            'genres': [genre['name'] for genre in movie.get('genres', [])],
            'production_companies': [company['name'] for company in movie.get('production_companies', [])]
        }
    else:
        print(f"⚠️ Failed to fetch details for movie {movie_id}")
        return None

In [ ]:
movies_list = fetch_movies(max_pages=500)  # Fetch up to 10,000 movies

for movie in movies_list:
    details = fetch_movie_details(movie['movie_id'])
    if details:
        movie.update(details)  # Add extra movie details

print(f"✅ Total movies fetched: {len(movies_list)}")

Fetched 1000 movies so far...
Fetched 2000 movies so far...
Fetched 3000 movies so far...
Fetched 4000 movies so far...
Fetched 5000 movies so far...
Fetched 6000 movies so far...
Fetched 7000 movies so far...
Fetched 8000 movies so far...
Fetched 9000 movies so far...
Fetched 10000 movies so far...
✅ Total movies fetched: 10000


In [ ]:
movies_df = pd.DataFrame(movies_list)

# Convert date to datetime format
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

# Display first 5 rows
movies_df.head()

,movie_id,title,release_date,vote_average,vote_count,popularity,overview,poster_url,budget,revenue,runtime,genres,production_companies
0,950396,The Gorge,2025-02-13,7.800,1699,974.107,Two highly trained operatives grow close from ...,https://image.tmdb.org/t/p/w500/7iMBZzVZtG0oBu...,0,0,127,"[Romance, Science Fiction, Thriller]","[Skydance Media, Crooked Highway, Apple Studios]"
1,1126166,Flight Risk,2025-01-22,6.000,391,925.912,A U.S. Marshal escorts a government witness to...,https://image.tmdb.org/t/p/w500/q0bCG4NX32iIEs...,25000000,40420193,91,"[Action, Thriller, Crime]","[Davis Entertainment, Icon Productions, Hammer..."
2,1064213,Anora,2024-10-14,7.099,1463,747.373,A young sex worker from Brooklyn gets her chan...,https://image.tmdb.org/t/p/w500/qh8m8Udz0sCa5g...,6000000,40989193,139,"[Drama, Comedy, Romance]","[Cre Film, FilmNation Entertainment]"
3,762509,Mufasa: The Lion King,2024-12-18,7.500,1566,651.915,"Mufasa, a cub lost and alone, meets a sympathe...",https://image.tmdb.org/t/p/w500/lurEK87kukWNaH...,200000000,700197856,118,"[Adventure, Family, Animation]",[Walt Disney Pictures]
4,1241982,Moana 2,2024-11-21,7.159,1788,635.733,After receiving an unexpected call from her wa...,https://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZ...,150000000,1056396565,99,"[Animation, Adventure, Family, Comedy]","[Walt Disney Pictures, Walt Disney Animation S..."


In [ ]:
movies_df.to_csv("movies_data.csv", index=False)
from google.colab import files
files.download("movies_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>